In [ ]:
import os, sys, re, glob, pandas, importlib
import numpy as np
sys.path.append('/Users/dp/pma/')
top = top_dir = '/Users/dp/pma/dataAndScripts/clip/meta/'
pma_dir = '/Users/dp/pma/'
os.chdir(top)
import sameRiver
import sameRiver.exp
importlib.reload(sameRiver.exp)

## Insert sizes for GEO upload. Not sure this is needed?

In [ ]:
sizes = {}
with open('tables/insert_sizes.txt') as f:
    for li in f:
        #print(li)
        s = li.rstrip('\n').split('\t')
        s[1] = '{' + s[1].split(' {')[1].rstrip(')')#re.sub(r"defaultdict(<class 'int'>, ", "", s[1])
        s[1] = dict(eval(s[1]))
        sizes[s[0]] = s[1]
#

rows = []
import matplotlib.pyplot as plt

for k, _dict in sizes.items():
    # _dict = {length: number_of_instances}
    lengths = np.fromiter(_dict.keys(), dtype=float)
    num_obs = np.fromiter(_dict.values(), dtype=float)
    #plt.plot(lengths, num_obs, 'k.')
    #plt.title(k)
    #plt.show()
    #plt.clf()
    #print(lengths)
    total_obs = np.sum(num_obs)
    mu = np.sum([n_obs*size for size, n_obs in zip(lengths, num_obs)])/total_obs
    var = np.average((lengths - mu)**2, weights=num_obs)
    #print(k, mu, np.sqrt(var))
    k = os.path.basename(k)
    spk = k.split('_')
    rows.append({
        'file name 1': k + '.gz',
        'file name 2': '_'.join([spk[0], spk[1], spk[2], 'R2', spk[3]]) + '.gz',
        'average insert size': np.around(mu, decimals=3),
        'standard deviation': np.around(np.sqrt(var), decimals=3),
    })
df = pandas.DataFrame(rows)

df = df.loc[:,['file name 1', 'file name 2', 'average insert size', 'standard deviation']]

df.to_excel('tables/insert_sizes_for_geo_upload.xlsx')

if os.path.exists('tables/insert_sizes_for_geo_upload.xlsx'):
    inserts = pandas.read_excel('tables/insert_sizes_for_geo_upload.xlsx')
    inserts = inserts.loc[[x in geo['raw file'].tolist() for x in inserts['file name 1']], :]
    inserts.to_excel('tables/insert_sizes_for_geo_upload_subset_to_those_included_in_upload.xlsx')

## Write an excel file of what datasets were used for what.
I think this is not necessary

In [ ]:
import sameRiver.scheme


from logs.datasets_included_in_positiveCounts_object import datasets_in_positiveCounts
from logs.datasets_used_for_biotypes import datasets_for_rbps, datasets_for_randos, datasets_for_pcbp1
from logs.datasets_included_in_negativeCounts_object import datasets_in_negativeCounts
from logs.datasets_used_for_heatmap import datasets_used_for_heatmap
datasets_in_biotypes = set(datasets_for_rbps + datasets_for_randos + datasets_for_pcbp1)
def where_am_i_used(name):
    used = {
        'positiveCounts': '', 'negativeCounts': '', 'biotypes': '', 'heatmap': ''}
    if name in datasets_in_negativeCounts:
        used['negativeCounts'] = name
    if name in datasets_in_positiveCounts:
        used['positiveCounts'] = name
    if name in datasets_in_biotypes:
        used['biotypes'] = name
    if name in datasets_used_for_heatmap:
        used['heatmap'] = name
    used['places_used'] = sum([(x!='') for x in used.values()])
    return used

scheme = sameRiver.scheme.scheme('./scheme.xlsx')
scheme.scheme_df['long_basename'] = [x.split('.fastq')[0] for x in scheme.scheme_df['long_fname_R1']]

rows = []
for row in scheme.scheme_df.to_dict('records'):
    row.update(where_am_i_used(row['long_basename']))
    rows.append(row)
used_datasets = pandas.DataFrame(rows)
used_datasets.to_excel('tables/where_datasets_were_used.xlsx')
#print(used_datasets[used_datasets['places_used']==0])
#print(scheme.scheme_df)

## Renaming some Rbfox1 files.
Not necessary anymore.

In [ ]:
import re, os, pandas
def rename_split_r1r2_dir_based_on_scheme(exp_name, prefix, scheme_fname, dir_name, out_dir):
    scheme = pandas.read_excel(scheme_fname)
    scheme['fname_R1'] = [
        '{d}/{p6}/{p6}_{p3}.fastq'.format(d=dir_name, p6=p6_bc, p3=p3_bc) \
        for (p6_bc, p3_bc) in zip(scheme.P6_BC, scheme.P3_BC)]  
    scheme['fname_R2'] = [
        '{d}/{p6}/{p6}_R2_{p3}.fastq'.format(d=dir_name, p6=p6_bc, p3=p3_bc) \
        for (p6_bc, p3_bc) in zip(scheme.P6_BC, scheme.P3_BC)]

    scheme['upload_fname_R1'] = [
    '{o}/{exp}_{gene}_{prefix}{p6}_{p3}.fastq'.format(
        o=out_dir, prefix=prefix,
        exp=exp_name, gene=re.sub(':', '-', gene), p6=p6, p3=p3) for (gene, p6, p3) in zip(
            scheme['Gene'], scheme['P6_BC'], scheme['P3_BC'])
    ]
    scheme['upload_fname_R2'] = [
    '{o}/{exp}_{gene}_{prefix}{p6}_R2_{p3}.fastq'.format(
        o=out_dir, prefix=prefix,
        exp=exp_name, gene=re.sub(':', '-', gene), p6=p6, p3=p3) for (gene, p6, p3) in zip(
            scheme['Gene'], scheme['P6_BC'], scheme['P3_BC'])
    ]
    print(scheme.iloc[0])
    print(scheme['upload_fname_R1'].tolist())
    
    def try_moving(infile, outfile):
        if not os.path.exists(os.path.dirname(outfile)):
            os.system('mkdir ' + os.path.dirname(outfile))
        if not os.path.exists(infile):
            print("Not found: {}".format(infile))
        else:
            if os.path.exists(outfile):
                print("Exists (skipping): {}".format(outfile))
                return
            cmd = 'rsync {} {}'.format(infile, outfile)
            print(cmd)
            os.system(cmd)

    for row in scheme.to_dict('records'):
        try_moving(row['fname_R1'], row['upload_fname_R1'])
        try_moving(row['fname_R2'], row['upload_fname_R2'])
        
        
rename_split_r1r2_dir_based_on_scheme(
    'Exp73', '73', '/Users/dfporter/clip/miseq/Runs/190413rbfox/scheme.xlsx',
    '/Users/dfporter/clip/miseq/Runs/190413rbfox/fastq/r1r2_split/',
    '/Users/dfporter/clip/miseq/Runs/190413rbfox/fastq/r1r2_split_renamed/')

In [ ]:
pcbp1s = [
    'PCBP1',
    'PCBP1 100P', 'PCBP1 100Q', 'PCBP1 dKH',
    'PCBP1:100P', 'PCBP1:100Q','PCBP1:dKH',
    'PCBP1-100P', 'PCBP1-100Q','PCBP1-dKH',
]
randos = [
    'UBA2', 'ETS2', 'CAPNS6', 'TPGS2', 
    'EPB41L5', 'CHMP3', 'CDK4', 'IDE', 
    'DCTN6', 'CCIN', 'ITPA', 
]
rbps_endog = ['FBL', 'hnRNPC', 'Rbfox2']
uORF = ['CELF1', 'Rbfox1',  'hnRNPD']
pSLX3 = [
    'NUFIP', 'NUFP1', 'NUFIP-R474W', 'NUFIP1-R474W',
    'DICER1', 'DICER1-R944Q',
    'CNOT9', 'CNOT9-P131L',
]
pSLX4 = [
    'RBFOX1', 'RBFOX1-A69T',
    'RARS2', 'RARS2-R6C',
    'RPL5', 'RPL5-E82K',
]
pSLX3 = [
    'SMAD4', 'SMAD4-R361H',
    'SMAD3',
    'CRNKL1', 'CRNKL1-S128F',
    'BRCA1', 'BARD1',
]
NpL = [
    'RBM11', 'RBM11-R74C',
    'A1CF', 'A1CF-E34K',
    'NOVA1', 'NOVA1-R146C',
    'RBM39', 'RBM39-T353I',
    'FUBP1', 'FUBP1-R429C',
    'CNOT1', 'CNOT1-R499H',
    'KHDRBS2', 'KHDRBS2-R168C',
    # ...
    'DDX3X', 'DDX3X-R528C',
]
#    'KPNB1', 'KPNB1-L552F'
#rbps_pLEX = ['SF3B1']

def to_expression_type(_type):
    #print(_type)
    if _type in pcbp1s:
        return 'FLAG-HA-HIS tagged ORF CRISPR targetted to AAVS1 safe harbor locus with Puro selection'
    if _type in randos:
        return 'FLAG-HA-HIS tagged ORF transiently expressed from pLEX vector 2 days without selection'
    if _type in NpL:
        return 'FLAG-HA-HIS tagged ORF transiently expressed from pLEX vector 1-2 days without selection'
    if _type in rbps_endog:
        return 'Immunopurified endogenous protein (unmodified cells)'
    if _type in pSLX3:
        return 'N-terminally FLAG-HA-HIS tagged ORF expressed from a pLEX-derived vector (pSLX3).'
    if _type in pSLX4:
        return 'C-terminally FLAG-HA-HIS tagged ORF expressed from a pLEX-derived vector (pSLX4).'
    if _type in uORF:
        return 'FLAG-HA-HIS tagged ORF expressed from pLEX with a uORF in front of CDS to lower expression levels'
    if _type == 'HCT116':
        return 'Unmodified cells, immunopurified with anti-HA but without epitope'
    if re.sub('[:-]', ' ', _type) in ['SF3B1', 'SF3B1 K700E']:
        return '3X FLAG tagged SF3B1 expressed from pLEX vector for 2 days without selection'
    if 'STD' in _type:
        return 'cDNA standard for library quantification.'
    if _type == 'No-vector':
        return 'Unmodified cells, not expressing any epitope.'
    #if _type in ['100Pox', '100Qox', 'PCBP1ox']:
    print(f"Couldn't find {_type}")
    return

File 2 Description of datasets currently has the headers:
```text
    title	source name	Expression	L3-BC	L5-BC	raw file	raw file read mate	Read 1 file checksum	instrument model	read length	single or paired-end

    easyCLIP CCIN Replicate 1	HEK293T cells	FLAG-HA-HIS tagged ORF transiently expressed from pLEX vector 2 days without selection	TCA	CTGATC	Exp56_CCIN_tcCTGATC_TCA.fastq.gz	Exp56_CCIN_tcCTGATC_R2_TCA.fastq.gz	1d3ad6fbda45862d06caa07f30c930a4	Illumina Miseq	76	paired-end
```
    

In [ ]:
import collections
from sameRiver.readsPerGene import *

os.chdir(top)

geo_upload_columns = [
    'Sample name', 'title', 'source name', 'organism',
    'characteristics: Expression',
    'characteristics: Exp',
    'characteristics: L3-BC', 
    'characteristics: L5-BC',
    'molecule', 'description', 'processed data file', 'raw file']

scheme_fname = f'{top}/scheme.xlsx'
geo_fname = f'{pma_dir}/doc/seq_template_v2.1.xls'

# Just to use the blacklist and column names.
rpg = readsPerGene(f'{top}/ann_counts.txt')
black_list = rpg.set_blacklist()

# Read the scheme to serve as a template for the run information table.
scheme = pandas.read_excel(scheme_fname)
print('>>>', len(scheme.index))
scheme.drop_duplicates(subset='long_fname_R1', inplace=True)
print(len(scheme.index))
scheme['Gene'] = [re.sub(':', '-', x) for x in scheme.Gene]

# Make the run information dataframe and give it a numeric index.
geo = scheme.copy()
geo['Sample name'] = range(1, len(geo.index)+1)

# Add information to the table.
geo['source name'] = [
    {True: 'HCT116 cells', False: 'HEK293T cells'}[
        bool('PCBP1' in protein or 'HCT116' in protein)] for protein in geo['Gene']]
geo['organism'] = 'H. sapiens'
geo['characteristics: Expression'] = [to_expression_type(x) for x in geo['Gene']]
geo['characteristics: Exp'] = geo['Experiment']
geo['characteristics: Protein'] = geo['Gene']
geo['characteristics: L3-BC'] = geo['P3_BC']
geo['characteristics: L5-BC'] = [x for x in geo['P6_BC']]
geo['molecule'] = 'Library from RNA UV cross-linked to the indicated protein'
geo['processed data file'] = 'ann_counts.txt'
geo['basename'] = ['_'.join([str(_) for _ in x]) for x in zip(
    geo['Experiment'], geo['Gene'], geo['P6_BC'], geo['P3_BC'])]
geo['basename_r2'] = ['{}_{}_{}_R2_{}'.format(*x) for x in zip(
    geo['Experiment'], geo['Gene'], geo['P6_BC'], geo['P3_BC'])]

def purged(name):
    """
    black_list = [
        'Exp16_FBL_24AGCTAG_CAG',  # Very small dataset.
        'Exp16_FBL_24AGCTAG_AAC',  # I believe this was also discarded because it was too small.
        'Exp16_hnRNPC_24TGAGTG_AGT', # Unknown why this was blacklisted.
        'Exp16_hnRNPC_24TGAGTG_CAG', # Unknown why this was blacklisted.
        'Exp28_hnRNPC_17CGATTA_AAC', # Unknown why this was blacklisted.
        'Exp31_UBA2_15TGAGTG_AAC',# Too correlated with CDK4 AAC Exp31.
        'Exp31_UBA2_15TGAGTG_CAG', # Empty.
        'Exp31_UBA2_05TGAGTG_CAG',
        'Exp31_CDK4_15GCCATG_AAC', # Too correlated with UBA2 AAC Exp31.
        'Exp31_CDK4_15GCCATG_CAG', # Empty.
        'Exp33_CDK4_17GCCATG_AGT',
        'Exp31_CDK4_05GCCATG_CAG',
        'Exp33_CAPNS6_17CACTGT_TCA', # Empty.
        'Exp61_PCBP1-100P_hpGCCATG_TCA',  # Too small.
        'Exp61_PCBP1-100P_hpGCCATG_AGT',  # Too small.
        ]"""
    
    #unused_datasets = used_datasets.loc[[x<1 for x in used_datasets['places_used']], :]['long_basename'].tolist()
    #unused_datasets = [x for x in unused_datasets if not re.search('No[_ ]vector', x, re.IGNORECASE)]

    #black_list.extend(unused_datasets)
    if 'BRCA' in name:
        print(f"checking {name}")
    if name not in rpg.df.columns:
        print("Not in rpg.df.columns")
        return False
    if name in rpg.black_list:
        return False
    if 'STD' in name:
        return False
    if 'BRCA' in name:
        print(f"Found {name}")
    return True

print(f"{[x for x in geo['basename'] if 'BRCA' in x]}")
print(f"{[x for x in rpg.df.columns if 'BRCA' in x]}")
print(f"Before purge: {len(geo.index)} datasets.")
geo = geo.loc[[purged(name) for name in geo['basename']], :]
print(f"After purge: {len(geo.index)} datasets.")
print(f"{[x for x in geo['basename'] if 'BRCA' in x]}")

rep_n = {}
def add_replicate(name):
    if name in rep_n:
        rep_n[name] += 1
    else:
        rep_n[name] = 1
    return 'easyCLIP {} Replicate {}'.format(name, rep_n[name])

geo['title'] = [add_replicate(protein) for protein in geo['Gene']]
geo['raw file'] = ['{}.fastq.gz'.format(x) for x in geo.basename]
geo['raw file read mate'] = ['{}.fastq.gz'.format(x) for x in geo.basename_r2]

geo_upload_columns += ['raw file read mate']
geo = geo.loc[:, [(x in geo_upload_columns) for x in geo.columns]]
geo = geo.loc[:, geo_upload_columns]

geo.index = geo['Sample name']
geo.to_excel(f'{top}/tables/seq_template_generated_sample_list.xlsx')
geo.to_excel(f'{top}/tables/File 2 Description of datasets.xlsx')

df = pandas.read_excel(f'{top}/tables/seq_template_generated_sample_list.xlsx')

fastq_unzip_dir = 'geo_upload_prezip/'
fastq_zip_dir = 'geo_upload/'
server_directory = './kkunqu/'
password = '33%9uyj_fCh?M16H'
upload_commands = open('upload_commands.sh', 'w')
zip_commands = open('zip_commands.sh', 'w')
for fname1_unzip, fname2_unzip in zip(df['raw file'], df['raw file read mate']):
    
    fname1_unzip = fastq_unzip_dir + fname1_unzip.rstrip('.gz')
    fname2_unzip = fastq_unzip_dir + fname2_unzip.rstrip('.gz')
    fname1 = fastq_zip_dir + '/' + os.path.basename(fname1_unzip) + '.gz'
    fname2 = fastq_zip_dir + '/' + os.path.basename(fname2_unzip) + '.gz'    

    zip_commands.write(f'gzip < {fname1_unzip} > {fname1} \n')
    zip_commands.write(f'gzip < {fname2_unzip} > {fname2} \n')
    for local_file_or_dir in [fname1, fname2]:
        if not os.path.exists(local_file_or_dir):
            #print("Could not find file {} to upload.".format(local_file_or_dir))
            pass
        
        cmd = "ncftpput -F -R -z -u geo "
        cmd += '-p "{}" ftp-private.ncbi.nlm.nih.gov '.format(password)
        cmd += ' {} {} '.format(server_directory, local_file_or_dir)
        upload_commands.write(cmd +'\n')
        
upload_commands.close()
zip_commands.close()


In [ ]:

geo = pandas.read_excel(geo_fname, sheet_name='Samples')

geo['geo_id'] = [','.join([str(x).split('.')[0] for x in arr]) for arr in zip(
    geo['characteristics: L3-BC'], geo['characteristics: L5-BC'], geo['characteristics: Run_P6BC'])]

desired_geo_ids = set(geo.geo_id)
scheme['geo_id'] = [','.join([str(x).split('.')[0] for x in arr]) for arr in zip(
    scheme['L3'], scheme['L5'], scheme['P6_BC'])]
geo['Basename'] = [ '_'.join(arr) for arr in zip(
    geo['characteristics: Exp'], geo['characteristics: L3-BC'], geo['characteristics: L5-BC'], geo['characteristics: Run_P6BC']
)]
scheme = scheme[[(x in desired_geo_ids) for x in scheme.geo_id]]

to_dir = {
    '05': '/Volumes/Seagate/hiseq/180105/fastq/r1r2_split/',
    '17': '/Volumes/Seagate/hiseq/180117/fastq/r1r2_split/',
    'tc': '/Volumes/Seagate/Miseq/Runs/181115/fastq/r1r2_split/',
    'pc': '/Users/dfporter/pma/dataAndScripts/clip/miseq/Runs/190113/fastq/r1r2_split/',
    '24':'/Volumes/Seagate/hiseq/170924/fastq/r1r2_split/',
}

scheme['dir'] = [to_dir[p6bc[:2]] for p6bc in scheme.P6_BC]

scheme['fname_R1'] = [
    '{d}/{p6}/{p6}_{p3}.fastq'.format(d=_dir, p6=p6_bc[2:], p3=p3_bc) \
    for (_dir, p6_bc, p3_bc) in zip(scheme.dir, scheme.P6_BC, scheme.P3_BC)]

scheme['fname_R2'] = [
    '{d}/{p6}/{p6}_R2_{p3}.fastq'.format(d=_dir, p6=p6_bc[2:], p3=p3_bc) \
    for (_dir, p6_bc, p3_bc) in zip(scheme.dir, scheme.P6_BC, scheme.P3_BC)]

scheme['upload_fname_R1'] = [
    '{o}/{exp}_{gene}_{p6}_{p3}.fastq'.format(
        o='/Volumes/Seagate/hiseq/to_upload/',
        exp=exp, gene=re.sub(':', '-', gene), p6=p6, p3=p3) for (exp, gene, p6, p3) in zip(
            scheme['Experiment'], scheme['Gene'], scheme['P6_BC'], scheme['P3_BC'])
]
scheme['upload_fname_R2'] = [
    '{o}/{exp}_{gene}_{p6}_R2_{p3}.fastq'.format(
        o='/Volumes/Seagate/hiseq/to_upload/',
        exp=exp, gene=re.sub(':', '-', gene), p6=p6, p3=p3) for (exp, gene, p6, p3) in zip(
            scheme['Experiment'], scheme['Gene'], scheme['P6_BC'], scheme['P3_BC'])
]
#print(scheme.upload_fname_R1)

def try_moving(infile, outfile):
    if not os.path.exists(infile):
        print("Not found: {}".format(infile))
    else:
        if os.path.exists(outfile):
            print("Exists (skipping): {}".format(outfile))
            return
        cmd = 'rsync {} {}'.format(infile, outfile)
        print(cmd)
        os.system(cmd)

#for row in scheme.to_dict('records'):
#    try_moving(row['fname_R1'], row['upload_fname_R1'])
#    try_moving(row['fname_R2'], row['upload_fname_R2'])



In [ ]:
# RAW FILES table
# GEO columns: 

md5 = pandas.read_csv(f'{top}/tables/md5values.txt', sep=' ')
md5['file_checksum'] = md5.index
md5['raw_file'] = [os.path.basename(x) for x in md5.raw_file]
to_md5 = dict(zip(md5['raw_file'], md5['file_checksum']))


run_code_to_instrument_and_read_len = {
    'mg': {'Instrument': 'Illumina Hiseq 2500+Illumina Miseq', 'read length': 0},
    'rb': {'Instrument': 'Illumina Hiseq 4000', 'read length': 76},
    'hp': {'Instrument': 'Illumina Hiseq 4000', 'read length': 76},
    '05': {'Instrument': 'Illumina Hiseq 2500', 'read length': 125},
    '17': {'Instrument': 'Illumina Hiseq 2500', 'read length': 125},
    '24': {'Instrument': 'Illumina Hiseq 2500', 'read length': 125},
    'pc': {'Instrument': 'Illumina Miseq', 'read length': 76},
    'tc': {'Instrument': 'Illumina Miseq', 'read length': 76},
}

def fname_to_sequencing_run(fname):
    s = os.path.basename(fname).split('_')
    run_code = s[2][:2]
    return run_code_to_instrument_and_read_len[run_code]['Instrument']

def fname_to_read_length(fname):
    s = os.path.basename(fname).split('_')
    run_code = s[2][:2]
    return run_code_to_instrument_and_read_len[run_code]['read length']

df = pandas.read_excel('tables/seq_template_generated_sample_list.xlsx', index_col=0)

rows = []
for row in df.to_dict('records'):
    rows.append({
        'file name': row['raw file'],
        'file type': 'fastq',
        'instrument model': fname_to_sequencing_run(row['raw file']),
        'read length': fname_to_read_length(row['raw file']),
        'single or paired-end': 'paired-end',
        'file checksum': to_md5.get(row['raw file'], '')
    })
    rows.append({
        'file name': row['raw file read mate'],
        'file type': 'fastq',
        'instrument model': fname_to_sequencing_run(row['raw file read mate']),
        'read length': fname_to_read_length(row['raw file']),
        'single or paired-end': 'paired-end',
        'file checksum': to_md5.get(row['raw file read mate'], '')
    })
df = pandas.DataFrame(rows)
df = df.loc[:, ["file name", "file type", "file checksum", "instrument model", "read length", "single or paired-end"]]

df.to_excel('tables/raw_files_table_for_geo_upload.xlsx')